In [ ]:
import apache_beam as beam


def split_row(r): return r.split(',')
def filter_by_state(s): return lambda r: r[4] == s


class MyPTransform(beam.PTransform):
    def expand(self, input_or_inputs):
        return (
            input_or_inputs
            | "Map California coords" >> beam.Map(self.map_coords)
            | "Output California" >> beam.Map(self.print_output)
        )
    # we could do all at once, lack of creativity right now...
    def map_coords(self, r): return [r[0], r[4], "{0}:{1}".format(r[6], r[7])]
    def print_output(self, r): print("{0} ({1}): {2}".format(r[0], r[1], r[2]))


p = beam.Pipeline()

input_pcollection = (p
                     | beam.io.ReadFromText("../../data/data.csv", skip_header_lines=1)
                     | beam.Map(split_row))

california_pipeline = (
    input_pcollection
    | "Filter by California" >> beam.Filter(filter_by_state("California"))
    | "Output California" >> MyPTransform()
)

virginia_pipeline = (
    input_pcollection
    | "Filter by Virginia" >> beam.Filter(filter_by_state("Virginia"))
    | "Output Virginia" >> MyPTransform()
)

p.run()
